In [1]:
DATA_NAME = 'spaceNet-full' 
TRANSFORM = 'gabor'
CHANNEL = 'blue'
PARAM_CSV = 'gabor_new.csv' # only use for Gabor

In [2]:
path_list = DATA_NAME.split("-") + TRANSFORM.split("-")
if CHANNEL:
    path_list.append(CHANNEL)
print(f"Name the notebook:\nindependence_{'_'.join(path_list[::-1])}.ipynb")
FULL_DATA_NAME='-'.join(path_list)


import git
from pathlib import Path
import os
CWD = os.getcwd()
ROOT_DIR = Path(git.Repo('.', search_parent_directories=True).working_tree_dir)

Path(os.path.join(CWD, "CSVs")).mkdir(exist_ok=True)
Path(os.path.join(CWD, "plots")).mkdir(exist_ok=True)
Path(os.path.join(CWD, "cache")).mkdir(exist_ok=True)

GROUP = 'layer' if TRANSFORM.split("-")[0] == 'wavelet' else ('band' if TRANSFORM.split("-")[0] == 'fourier' else 'error')
RERUN = False
SKIP_OPTIMIZE_STEP = False

Name the notebook:
independence_blue_gabor_full_spaceNet.ipynb


In [3]:
os.chdir(os.path.join(ROOT_DIR, "utilities"))
from testing import * # If MATLAB is not installed, open utilities and set to False
from plotting import *
os.chdir(CWD)
np.random.seed(0)

In [4]:
from scipy import spatial
from sklearn.decomposition import PCA

In [5]:
group_data_map = pd.read_pickle(os.path.join(ROOT_DIR, "transformed-data", f'{FULL_DATA_NAME}.pickle'))
group_total_samples = pd.read_pickle(os.path.join(ROOT_DIR, "transformed-data", f'{FULL_DATA_NAME}-size.pickle'))
param_df = pd.read_csv(os.path.join(ROOT_DIR, "gabor", PARAM_CSV))

In [6]:
if 'fourier' in TRANSFORM:
    GROUPS = np.arange(2, sorted(group_data_map)[-1] + 1)#[::3]
elif 'wavelet' in TRANSFORM:
    GROUPS = np.arange(2, sorted(group_data_map)[-1] + 1)
elif 'learned' in TRANSFORM:
    nonskewed_df = pd.read_csv(os.path.join(ROOT_DIR, 'learned-filters', 'nonskewed_filter_idxs_df.csv')).set_index(['dataset', 'num_images', 'num_bootstrap'])
    nonskewed_filter_idxs = eval(nonskewed_df.loc[DATA_NAME].sort_values('num_images', ascending=False)['nonskewed_filter_idxs'].iloc[0]) 
    GROUPS = nonskewed_filter_idxs # can set to filter_group_map.keys() to include all prepared filters
elif 'gabor' in TRANSFORM:
    GROUPS = param_df['index']

In [7]:
n_bootstrap = int(1e5) 
bootstrap_size = int(1e4)


In [8]:

cov_matrix = np.zeros((len(GROUPS), len(GROUPS)))
for _ in tqdm(range(n_bootstrap)):
    X = np.zeros((bootstrap_size, len(GROUPS)))
    for i in range(len(GROUPS)):
        group = GROUPS[i]
        if len(group_data_map[group]) < 100:
            X[:, i] = np.full(bootstrap_size, np.nan)
        else:
            X[:, i] = np.random.choice(group_data_map[group], size=(bootstrap_size), replace=True)
    cov_matrix += np.cov(X, rowvar=False)
cov_matrix /= n_bootstrap

  0%|          | 0/100000 [00:00<?, ?it/s]

In [9]:



# Convert covariance matrix to DataFrame for better formatting
cov_df = pd.DataFrame(cov_matrix, index=GROUPS, columns=GROUPS)
cov_df = cov_df.dropna(how='all').dropna(axis=1, how='all')
cov_df.to_csv(os.path.join(CWD, "CSVs", f'covariance_matrix.csv'))
cov_df.round(2)

index,0,1,2,3,4,5,6,7,8,9,...,32,33,34,35,36,37,38,39,40,41
index,,,,,,,,,,,,,,,,,,,,,
0,104.73,0.0,-0.00,0.00,0.00,-0.00,-0.00,-0.01,0.00,-0.00,...,-0.0,0.0,-0.00,0.00,-0.00,-0.00,0.00,0.00,0.00,0.00
1,0.00,44.7,0.00,-0.00,-0.00,-0.00,-0.00,0.00,-0.00,-0.00,...,-0.0,-0.0,0.00,0.00,-0.00,-0.00,-0.00,0.00,0.00,0.00
2,-0.00,0.0,19.63,-0.00,0.00,0.00,0.00,0.00,0.00,-0.00,...,0.0,-0.0,0.00,-0.00,0.00,0.00,-0.00,-0.00,0.00,-0.00
3,0.00,-0.0,-0.00,7.42,-0.00,0.00,-0.00,-0.00,-0.00,0.00,...,-0.0,-0.0,0.00,0.00,0.00,0.00,0.00,0.00,-0.00,-0.00
4,0.00,-0.0,0.00,-0.00,3.18,-0.00,0.00,-0.00,-0.00,0.00,...,0.0,0.0,0.00,0.00,0.00,-0.00,0.00,0.00,0.00,-0.00
5,-0.00,-0.0,0.00,0.00,-0.00,0.92,0.00,-0.00,0.00,-0.00,...,-0.0,-0.0,0.00,-0.00,0.00,-0.00,0.00,0.00,0.00,-0.00
6,-0.00,-0.0,0.00,-0.00,0.00,0.00,0.58,0.00,0.00,0.00,...,0.0,-0.0,-0.00,-0.00,-0.00,0.00,-0.00,0.00,-0.00,0.00
7,-0.01,0.0,0.00,-0.00,-0.00,-0.00,0.00,135.91,-0.00,0.00,...,0.0,0.0,-0.00,0.00,-0.00,-0.00,-0.00,0.00,0.00,-0.00
8,0.00,-0.0,0.00,-0.00,-0.00,0.00,0.00,-0.00,58.52,0.00,...,-0.0,0.0,0.00,0.00,0.00,-0.00,0.00,0.00,-0.00,-0.00


In [10]:
corr_matrix = cov_df / np.sqrt(np.outer(np.diag(cov_df), np.diag(cov_df)))
corr_matrix.round(5)

index,0,1,2,3,4,5,6,7,8,9,...,32,33,34,35,36,37,38,39,40,41
index,,,,,,,,,,,,,,,,,,,,,
0,1.00000,0.00005,-0.00001,0.00004,0.00001,-0.00004,-0.00003,-0.00004,0.00002,-0.00002,...,-0.00003,0.00008,-0.00001,0.00002,-0.00000,-0.00003,0.00001,0.00001,0.00001,0.00002
1,0.00005,1.00000,0.00007,-0.00006,-0.00002,-0.00005,-0.00001,0.00004,-0.00001,-0.00003,...,-0.00000,-0.00000,0.00000,0.00003,-0.00002,-0.00001,-0.00004,0.00001,0.00002,0.00003
2,-0.00001,0.00007,1.00000,-0.00001,0.00000,0.00004,0.00004,0.00001,0.00004,-0.00002,...,0.00002,-0.00003,0.00003,-0.00004,0.00005,0.00000,-0.00007,-0.00000,0.00005,-0.00000
3,0.00004,-0.00006,-0.00001,1.00000,-0.00003,0.00003,-0.00001,-0.00004,-0.00003,0.00001,...,-0.00003,-0.00004,0.00001,0.00004,0.00000,0.00003,0.00003,0.00002,-0.00003,-0.00000
4,0.00001,-0.00002,0.00000,-0.00003,1.00000,-0.00002,0.00001,-0.00000,-0.00001,0.00006,...,0.00000,0.00002,0.00000,0.00000,0.00003,-0.00003,0.00004,0.00002,0.00004,-0.00005
5,-0.00004,-0.00005,0.00004,0.00003,-0.00002,1.00000,0.00000,-0.00006,0.00003,-0.00001,...,-0.00002,-0.00003,0.00005,-0.00003,0.00000,-0.00000,0.00004,0.00000,0.00002,-0.00002
6,-0.00003,-0.00001,0.00004,-0.00001,0.00001,0.00000,1.00000,0.00002,0.00000,0.00005,...,0.00004,-0.00001,-0.00002,-0.00001,-0.00000,0.00002,-0.00002,0.00004,-0.00003,0.00001
7,-0.00004,0.00004,0.00001,-0.00004,-0.00000,-0.00006,0.00002,1.00000,-0.00000,0.00002,...,0.00000,0.00002,-0.00004,0.00001,-0.00005,-0.00002,-0.00004,0.00004,0.00003,-0.00003
8,0.00002,-0.00001,0.00004,-0.00003,-0.00001,0.00003,0.00000,-0.00000,1.00000,0.00002,...,-0.00003,0.00001,0.00001,0.00002,0.00004,-0.00006,0.00003,0.00003,-0.00001,-0.00002


In [11]:
np.linalg.norm(cov_df - np.diag(np.diag(cov_df)))  # Remove diagonal elements for visualization

0.03461966089718708

In [12]:
pca = PCA()
pca.fit(cov_df)

print("Singular values (explained variance):")
print(pca.explained_variance_)

print("\nPrincipal components (eigenvectors):")
eigenvectors = pca.components_
eigenvectors_df = pd.DataFrame(eigenvectors)
eigenvectors_df

Singular values (explained variance):
[4.41648298e+02 2.96791313e+02 2.65238497e+02 2.58098469e+02
 2.43740037e+02 2.13705703e+02 8.16092975e+01 5.96291309e+01
 5.24624946e+01 5.06583503e+01 4.69023701e+01 4.13563954e+01
 1.94091270e+01 1.33990186e+01 9.84960990e+00 9.60160153e+00
 9.18431793e+00 8.02914642e+00 2.95812839e+00 2.14787816e+00
 2.00733903e+00 1.63327562e+00 1.44607585e+00 1.21322589e+00
 4.10081482e-01 3.43435563e-01 2.88686768e-01 2.51049742e-01
 2.39879404e-01 1.84529082e-01 5.54162031e-02 3.71689034e-02
 2.89871760e-02 2.35860248e-02 2.15485388e-02 1.54263172e-02
 8.36103165e-03 6.16704805e-03 3.28461759e-03 2.72560599e-03
 2.38242406e-03 5.54666870e-29]

Principal components (eigenvectors):


,0,1,2,3,4,5,6,7,8,9,...,32,33,34,35,36,37,38,39,40,41
0,-0.039407,-0.007386,-0.002958,-0.001110,-0.000470,-0.000140,-0.000085,0.995907,-0.010660,-0.004422,...,-0.000459,-0.000146,-0.000048,-0.035984,-0.007839,-0.003052,-0.001399,-0.000512,-0.000163,-0.000049
1,-0.143340,-0.007167,-0.002724,-0.001016,-0.000432,-0.000121,-0.000080,0.034872,-0.010947,-0.004117,...,-0.000413,-0.000129,-0.000040,-0.100304,-0.007579,-0.002768,-0.001261,-0.000481,-0.000152,-0.000041
2,0.891959,-0.003997,-0.001500,-0.000547,-0.000235,-0.000072,-0.000044,0.014493,-0.006261,-0.002309,...,-0.000236,-0.000066,-0.000022,-0.218946,-0.004292,-0.001554,-0.000685,-0.000258,-0.000080,-0.000021
3,0.148523,-0.002863,-0.001050,-0.000383,-0.000161,-0.000043,-0.000031,0.009470,-0.004509,-0.001596,...,-0.000163,-0.000052,-0.000018,-0.666817,-0.002996,-0.001092,-0.000478,-0.000192,-0.000059,-0.000016
4,0.352772,-0.018337,-0.006751,-0.002442,-0.001044,-0.000304,-0.000196,0.052761,-0.029286,-0.010265,...,-0.001017,-0.000322,-0.000104,0.664637,-0.019518,-0.006923,-0.003121,-0.001155,-0.000372,-0.000107
5,0.184319,-0.025601,-0.009097,-0.003292,-0.001408,-0.000408,-0.000255,0.054237,-0.042552,-0.013983,...,-0.001381,-0.000444,-0.000135,0.228845,-0.027288,-0.009323,-0.004192,-0.001553,-0.000497,-0.000139
6,0.018146,-0.043860,-0.008745,-0.002993,-0.001261,-0.000360,-0.000230,0.011890,0.991390,-0.014972,...,-0.001237,-0.000394,-0.000121,0.018915,-0.051689,-0.009107,-0.003786,-0.001384,-0.000445,-0.000127
7,0.010885,-0.089380,-0.008472,-0.002745,-0.001181,-0.000337,-0.000212,0.007507,0.053278,-0.015663,...,-0.001151,-0.000353,-0.000116,0.011366,-0.145035,-0.008737,-0.003523,-0.001280,-0.000404,-0.000118
8,0.001877,-0.046405,-0.001664,-0.000555,-0.000222,-0.000073,-0.000043,0.001345,0.007065,-0.003408,...,-0.000236,-0.000070,-0.000027,0.001932,0.776830,-0.001841,-0.000706,-0.000265,-0.000079,-0.000023
9,0.010652,-0.511129,-0.010551,-0.003300,-0.001395,-0.000389,-0.000253,0.007532,0.039248,-0.020617,...,-0.001352,-0.000428,-0.000135,0.011023,0.503749,-0.010912,-0.004238,-0.001540,-0.000492,-0.000141


In [13]:
cos_dist = spatial.distance.cdist(eigenvectors, np.eye(cov_df.shape[0]), metric='cosine')
cos_dist =  [np.min(row) for row in cos_dist]
cos_dist


[0.0040933272191502335,
 0.024050602297922774,
 0.10804052964593669,
 0.27160093482016945,
 0.3353634647462187,
 0.08058011192888126,
 0.008609960897014268,
 0.027869157373003106,
 0.22317042953188404,
 0.3290964421608923,
 0.21502342573004762,
 0.13266207380804096,
 0.010486493974938949,
 0.017681661442703,
 0.20913704741917882,
 0.3512098189845356,
 0.28884934420088926,
 0.33515158717965965,
 0.016438392025269644,
 0.24067473959099317,
 0.2810253270454238,
 0.10500888382930518,
 0.1810648952686731,
 0.15779508381688823,
 0.040137772252313186,
 0.07155455509552477,
 0.09719864056429361,
 0.20579655497524263,
 0.2422471931432323,
 0.4106170940633428,
 0.031641456062050155,
 0.06413021015384412,
 0.10097035404873678,
 0.2097915484049001,
 0.2655730430761486,
 0.4020374910727511,
 0.2728828176621798,
 0.3821315956194007,
 0.17283193973437927,
 0.29456820056220023,
 0.23910783664351098,
 0.5286700555422791]